In [1]:
import pandas as pd
import json
import sqlite3

# 1. LOAD DATA
orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')

# Extract Restaurant data from SQL
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    conn.cursor().executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

# 2. MERGE DATA (The ETL Process)
# Join orders to users
step1 = pd.merge(orders, users, on='user_id', how='left')
# Join result to restaurants
final_df = pd.merge(step1, restaurants, on='restaurant_id', how='left', suffixes=('', '_master'))

# 3. EXPORT FINAL DATASET
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

# 4. QUICK ANALYSIS FOR ASSESSMENT ANSWERS
print(f"Total Rows: {len(final_df)}")
print(f"Distinct Users: {final_df['user_id'].nunique()}")
print(f"Gold Member AOV: {final_df[final_df['membership'] == 'Gold']['total_amount'].mean():.2f}")

Total Rows: 10000
Distinct Users: 2883
Gold Member AOV: 797.15
